In [ ]:
!pip install ucimlrepo

In [ ]:
!wget https://archive.ics.uci.edu/static/public/773/defungi.zip

--2024-12-11 05:55:15--  https://archive.ics.uci.edu/static/public/773/defungi.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘defungi.zip.1’

defungi.zip.1           [   <=>              ]  87.53M  9.66MB/s               

In [ ]:
!unzip defungi.zip -d /content/data/


In [ ]:
from torchvision import datasets, transforms

# Define transformations (you can modify these as needed)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224 (common for pretrained models)
    transforms.ToTensor(),          # Convert the images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet standards
])

# Create the dataset
DATA_DIR = '/content/data'

dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)

# Check class-to-index mapping
print("Class to Index Mapping:", dataset.class_to_idx)

# Check the length of the dataset and an example data point
print("Dataset size:", len(dataset))


In [ ]:
dataset[0]

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Extract a sample image and its label from the dataset
image, label = dataset[0]

def plot_image(image, label):
  # Reverse the normalization to display the image correctly
  unnormalize = transforms.Normalize(
      mean=[-0.485 / 0.229, -0.456 / 0.224, -0.406 / 0.225],
      std=[1 / 0.229, 1 / 0.224, 1 / 0.225]
  )

  image = unnormalize(image)

  # Convert the image tensor to a NumPy array and transpose it for plotting
  image = image.permute(1, 2, 0).numpy()

  # Plot the image using matplotlib
  plt.imshow(image)
  plt.title(f'Class label: {label}')
  plt.axis('off')  # Hide the axes
  plt.show()

plot_image(image, label)

In [ ]:
from torch.utils.data import random_split, DataLoader

# Calculate sizes for each split
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset into train, validation, and test sets
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders for each subset
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Define the CNN architecture
class BasicCNN(nn.Module):
    def __init__(self, num_classes=5, dropout_rate=0.5):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 26 * 26, 128)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(128, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return self.softmax(x)

# Hyperparameters
learning_rate = 0.005
num_epochs =20
dropout_rate = 0.2

# Initialize the model, loss function, and optimizer
model = BasicCNN(num_classes=5, dropout_rate=dropout_rate)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {running_loss/len(train_loader):.4f}, "
          f"Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {correct/total:.4f}")

# Testing
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")

Epoch 1/20, Train Loss: 1.4187, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 2/20, Train Loss: 1.4165, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 3/20, Train Loss: 1.4170, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 4/20, Train Loss: 1.4172, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 5/20, Train Loss: 1.4175, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 6/20, Train Loss: 1.4165, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 7/20, Train Loss: 1.4168, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 8/20, Train Loss: 1.4175, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 9/20, Train Loss: 1.4170, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 10/20, Train Loss: 1.4175, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 11/20, Train Loss: 1.4170, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 12/20, Train Loss: 1.4170, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 13/20, Train Loss: 1.4170, Val Loss: 1.4325, Val Accuracy: 0.4737
Epoch 14/20, Train Loss: 1.4172, Val Loss: 1.4325, Val Accuracy: 0.4737
E

In [ ]:
# Testing
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {correct/total:.4f}")

Test Loss: 1.4242, Test Accuracy: 0.4792
